# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
from azureml.core import WorkSpace Experiment, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl.utilities import get_primary_metrics
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
import joblib 
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

import os

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
os.makedirs('results')

In [ ]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'hr-analytics-automl'

# Check workspace details
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

experiment=Experiment(ws, experiment_name)

In [ ]:
# Check for existing cluster. Otherwise, create new cluster
try:
    cluster = ComputeTarget(workspace=ws, name="project-cluster")
    print("Cluster exists")
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cluster = ComputeTarget.create(ws, "project-cluster", config)

cluster.wait_for_completion()

In [1]:
# Saving cleansed dataset
x.to_csv("dataset/train.csv")

datastore = ws.get_default_datastore()
datastore.upload(src_dir="dataset")

NameError: name 'x' is not defined

In [ ]:
train_data = Dataset.Tabular.from_delimited_files(path=[(datastore, ('dataset/train.csv'))])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
get_primary_metrics("classification")

In [3]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "task": "classification", 
    "primary_metric": "f1_score",
    "training_data": training_data
    "label_column_name": "target",
    "n_cross_validations": 5,
    "featurization": "auto",
    "max_cores_per_iteration": -1,
    "max_concurrent_iterations": 4
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

SyntaxError: invalid syntax (<ipython-input-3-4909dbdefaf6>, line 9)

In [ ]:
# TODO: Submit your experiment
experiment = Experiment(workspace=ws, name="auto_exp")
remote_run = experiment.submit(automl_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

In [ ]:
remote_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_auto_run, best_auto_model = remote_run.get_output()
best_auto_run._final_estimator

In [ ]:
#TODO: Save the best model
joblib.dump(best_auto_model, filename="results/automl_best_model.joblib")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
env = best_auto_run.get_environment()

entry_script='score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)

In [ ]:
model = best_run.register_model(model_name="employee_churn", 
                                           description='AutoML model')
inference_config = InferenceConfig(entry_script = entry_script, environment = env)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

In [ ]:
service = Model.deploy(ws, 'employee_churn_api', [model], inference_config, deployment_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
%run endpoint.py

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.get_logs()

In [ ]:
service.delete()